<table><tr><td height="250" width="500"><img src="https://www.opendataportal.at/wp-content/uploads/2016/09/WU-Wien-Logo.png"/ height="250" width="500"></td><td height="250" width="250"><img src="https://beta.ownyourdata.eu/wp-content/uploads/2018/02/logo512-300x300.png" height="250" width="250"/></td></tr></table>
<hr>
<h1><center>Data Science Lab: Own Your Data </center></h1>
<hr>
<h2><center>Notebook 1: Finding the user's home adress </center></h2>

<h5>Loading neccessary modules

In [1]:
!pip install folium
!pip install geopy
import pandas as pd
import json
from pprint import pprint
import folium
import datetime
import geopy
import geopy.geocoders
from geopy.geocoders import Nominatim
import numpy as np

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


<h5>Setting filepath

In [2]:
filepath = "./data/user_archive_091118"

<h5>File information

In [3]:
with open (filepath, "r") as file:
    all_data = json.load(file)
data = all_data["data"]
print("File is in json-format and contains the following keys:")
pprint(all_data.keys())
print("\nWe are interested in the data. It is a list of dictionaries and contains:")
print(set().union(*(d.keys() for d in data)))

File is in json-format and contains the following keys:
dict_keys(['user_id', 'name:', 'email', 'data'])

We are interested in the data. It is a list of dictionaries and contains:
{'Location', 'Tagebuchnotizen', 'Seismic', 'Pollenbelastung Austria-1120', 'Stimmung', 'Einstellungen'}


<h5>-  Loading GPS Data into a Pandas Dataframe and <br><br>-  Showing relevant columns for this notebook</h5>

In [4]:
for dictionary in data:
    if "Location" in dictionary.keys():
        gps_data = dictionary["Location"]
gps = pd.DataFrame(gps_data)
gps = gps[["latitude", "longitude", "timestamp", "altitude"]]
print(gps.head())

    latitude  longitude                 timestamp  altitude
0  48.182885  16.326133  2018-10-24T20:28:59.845Z       0.0
1  48.182885  16.326070  2018-10-24T19:38:12.308Z       0.0
2  48.182885  16.326133  2018-10-24T20:15:11.147Z       0.0
3  48.182885  16.326133  2018-10-24T20:46:19.257Z       0.0
4  48.182869  16.326044  2018-10-24T19:52:27.413Z       0.0


<h5>Converting values to floats

In [5]:
cols = ["latitude", "longitude", "altitude"]
gps[cols] = gps[cols].apply(pd.to_numeric, errors = "coerce")
print(gps.dtypes)

latitude     float64
longitude    float64
timestamp     object
altitude     float64
dtype: object


<h5>Data Set Information

In [6]:
in_format = "%Y-%m-%dT%H:%M:%S.%fZ"
newest = datetime.datetime.strptime(gps["timestamp"].max(),in_format)
oldest = datetime.datetime.strptime(gps["timestamp"].min(),in_format)
duration = newest-oldest
print ("The dataset contains {} GPS points collected over {} days.".format(len(gps),duration.days))

The dataset contains 1084 GPS points collected over 38 days.


<h5>Creating a Map with Markers of tracked GPS points for visualisation purposes

In [7]:
from folium.plugins import MarkerCluster
mittel = gps.median()
karte = folium.Map(location=[mittel[0], mittel[1]], zoom_start = "4")
marker_cluster = MarkerCluster().add_to(karte)
for i in range(0,len(gps)):
    folium.Marker([gps.iloc[i]["latitude"], gps.iloc[i]["longitude"]],
                  popup = gps.iloc[i]["timestamp"]).add_to(marker_cluster)
karte    

<h5> Finding the most frequent place at night

In basic terms, home is where a person sleeps. Therefore we check if the a point is between 9pm and 5am. <br> The following function returns True if a test time is between start and end, else it returns False.

Possible issue is difference in time zones if the user lives in a different time zone for an extended period of time. This issue will be discussed.

In [8]:
def TimeFrame(test):
    evening = datetime.time(21,0)
    morning = datetime.time(7,0)
    tester = test.time()
    return (tester >= evening or tester <= morning)

We apply this function to the dataframe and create a new dataframe which contains only the values recorded between 9pm and 5 am.

In [9]:
gps["at_night"] = gps.apply(
    lambda row: True if TimeFrame(datetime.datetime.strptime(row["timestamp"], in_format)) else False,
    axis=1
)

In [10]:
night_time = gps.loc[gps["at_night"] == True]

We now sort the data frames night_time and gps by the frequency of latitude and longitude pairs

In [11]:
frequencies_night = night_time.groupby(["latitude", "longitude"]).size().reset_index(name="count")
frequencies = gps.groupby(["latitude", "longitude"]).size().reset_index(name="count")
frequencies_night = frequencies_night.sort_values(by = "count", ascending=False)
frequencies = frequencies.sort_values(by = "count", ascending = False)
print ("Most frequent nightly coordinates:\n", frequencies_night.head(5))
print("\n", "-" *50, "\n")
print("Most frequent coordinates:\n", frequencies.head(5))

Most frequent nightly coordinates:
      latitude  longitude  count
1   47.036121  15.461757     53
3   47.036146  15.461533     49
31  48.182908  16.326158     29
26  48.182867  16.326070     29
17  48.182624  16.326290     28

 -------------------------------------------------- 

Most frequent coordinates:
       latitude  longitude  count
17   47.036121  15.461757     63
20   47.036146  15.461533     55
133  48.182867  16.326070     30
162  48.182908  16.326158     29
77   48.182624  16.326290     28


<h5>Calculating the distance between each point</h5><br>
<p>Many coordinate pairs point to the same address. Converting each coordinate pair to an address would take too much time and is very unreliable, because of regular connection timeouts. Therefore, we assume that coordinate pairs that are near enough to each other belong to the same address. The distance between the coordinate pairs is calculated using the <b>Haversine distance</b>.

1. The following code snippet transforms the dataframe into a list of lists containing the latitude, longitude and count. I will refer to the list of lists as a matrix with rows for easier understanding.<br><br>
2. Starting in the first row (most frequent), the Haversine distances to all other rows are calculated. If the distance is smaller than 100 meters (some GPS values are not very accurate), the code assumes that these point to the same address. The counts are added and the less frequent row gets removed from the list.<br><br>
3. Finally the list is loaded back into a dataframe. <br><br><br>
The data was transformed to a list for easier handling.
  

In [12]:
def haversine_np(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return (km)


In [13]:
def list_reduction(dataframe):
    listformat = dataframe.values.tolist()
    c = 0
    for i, element in enumerate(listformat):
        othervalues = listformat[i+1:]
        for comparator in othervalues:
            distance = haversine_np(element[1],element[0], comparator[1], comparator[0])
            if distance <= 0.050:
                c += 1
                if c%9 == 0:
                    print("{:.4f} {:.4f} is {:.0f} meters away from {:.4f} {:.4f}.".format(element[0], element[1], distance*1000, comparator[0], comparator[1]))
                element[2] += comparator [2]
                listformat.remove(comparator)
            
    print("{} coordinate points were seen as equal".format(c))   
    print("\n", "-" * 50)
    labels = ["latitude", "longitude", "count"]
    output = pd.DataFrame.from_records(listformat, columns = labels)
    return(output)

In [14]:
gps_night = list_reduction(frequencies_night)
gps_allday = list_reduction(frequencies)

47.0361 15.4618 is 32 meters away from 47.0362 15.4613.
48.1829 16.3262 is 2 meters away from 48.1829 16.3262.
48.1829 16.3262 is 10 meters away from 48.1828 16.3261.
48.1829 16.3262 is 10 meters away from 48.1829 16.3263.
60.1684 24.9232 is 47 meters away from 60.1684 24.9223.
50 coordinate points were seen as equal

 --------------------------------------------------
47.0361 15.4618 is 37 meters away from 47.0362 15.4613.
48.1829 16.3261 is 8 meters away from 48.1829 16.3262.
48.1829 16.3261 is 27 meters away from 48.1831 16.3259.
48.1829 16.3261 is 13 meters away from 48.1829 16.3262.
48.1829 16.3261 is 5 meters away from 48.1829 16.3261.
48.1829 16.3261 is 9 meters away from 48.1829 16.3262.
48.1829 16.3261 is 15 meters away from 48.1827 16.3262.
48.1829 16.3261 is 14 meters away from 48.1828 16.3261.
48.1829 16.3261 is 7 meters away from 48.1829 16.3261.
48.1829 16.3261 is 19 meters away from 48.1827 16.3262.
48.1829 16.3261 is 14 meters away from 48.1828 16.3261.
48.1829 16.3261 

<h5>The dataframe is sorted by frequency

In [15]:
gps_night["count"] = gps_night["count"].astype(int)
gps_allday["count"] = gps_allday["count"].astype(int)
gps_night = gps_night.sort_values(by = "count", ascending = False).reset_index(drop=True)
gps_allday = gps_allday.sort_values(by = "count", ascending = False).reset_index(drop=True)

print("Most frequent nightly coordinates:\n", gps_night.head())
print("\n"*2, "-" * 50, "\n"*2)
print("Most frequent coordinates 24/7:\n", gps_allday.head())

Most frequent nightly coordinates:
     latitude  longitude  count
0  48.182908  16.326158    280
1  47.036121  15.461757    141
2  48.183065  16.327859     28
3  48.183547  16.324547     25
4  48.183313  16.323960     25


 -------------------------------------------------- 


Most frequent coordinates 24/7:
     latitude  longitude  count
0  48.182867  16.326070    448
1  47.036121  15.461757    167
2  48.183065  16.327859     41
3  48.212589  16.410448     33
4  48.183547  16.324547     25


<h5>Coordinate to address conversion</h5><br>
<p>For converting the coordinates to the address the Nominatim API is used

In [16]:
def get_address(lat, lon):
    geolocator = Nominatim(user_agent="just-searching-some-addresses")
    coordinate = ", ".join((str(lat),str(lon)))
    location = geolocator.reverse(coordinate, addressdetails = False)
    return(location.address)
def get_coordinates(x):
    geolocator = Nominatim(user_agent="reverse-searching-to-coordinates")
    location = geolocator.geocode(x)
    return(location)

<h5>The coordinates of the most frequent place are converted into the address</h5><br> For easier access the maximum values are stored in variables

In [17]:
lon_max_night, lat_max_night = gps_night.iloc[0]["longitude"], gps_night.iloc[0]["latitude"]
lon_max_allday, lat_max_allday = gps_allday.iloc[0]["longitude"], gps_allday.iloc[0]["latitude"]
wohnort_night = get_address(lat = lat_max_night, lon = lon_max_night)
wohnort_allday = get_address(lat = lat_max_allday, lon = lon_max_allday)
maximum_night, total_night = gps_night["count"].max(), gps_night["count"].sum()
maximum_allday, total_allday = gps_allday["count"].max(), gps_allday["count"].sum()
coordinates_address_night = get_coordinates(wohnort_night)
coordinates_address_allday = get_coordinates(wohnort_allday)

In [18]:
gps_allday["Distance from home"] = gps_allday.apply(lambda row: haversine_np(lat1 = lat_max_allday,lon1 = lon_max_allday,
                                                                             lat2 = row["latitude"], lon2 = row["longitude"]),
    axis=1
)
gps_night["Distance from home"] = gps_night.apply(lambda row: haversine_np(lat1 = lat_max_night,lon1 = lon_max_night,
                                                                             lat2 = row["latitude"], lon2 = row["longitude"]),
    axis=1
)

In [19]:
distance = 0.150
proximity_allday = gps_allday.loc[(gps_allday["Distance from home"] <= distance) & 
                                   (gps_allday["Distance from home"] > 0)]
proximity_night = gps_night.loc[(gps_night["Distance from home"] <= distance) &
                                (gps_night["Distance from home"] > 0)]
range_allday = proximity_allday["count"].sum()
range_night = proximity_night["count"].sum()

The Nominatim reverse address look-up returns the address string in different formats. Sometimes it includes districts, quarters and postbox. Therefore, this function should extract the important information from the string.

In [24]:
def dict_comprehension(in_str, maximum, total, proximity):
    global address
    in_list = in_str.split(", ")
    c = len(in_list)
    zip_code = None
    number = None
    complete = False
        
    # country is the last element in the list
    country = in_list[-1]
    # house number is the first element in the list, which starts with a number
    for i in in_list:
        if not number:
            try:
                int(i[0])
                number = i
                complete = True
            except:
                continue
    # street is the next element after the house number
    street = in_list[in_list.index(number)+1]
    # zip code is the last integer element in the list
    for i in reversed(in_list):
        if not zip_code:
            try:
                int(i)
                zip_code = in_list[-2]
            except:
                continue
        
    # province is the third element from the right. If the province is Vienna, the city name is Vienna too
    province = in_list[-3]
    if province == "Wien":
        city = "Wien"
    # otherwise city is to the left of the province information
    else:
        city = in_list[-4]
    
    # if there is only one string in the list, the list is not complete
    if number == zip_code or not complete:
        complete = False
        quarter = in_list[0]

    print("The GPS data was analyzed. {} out of {} ({:.2%}) GPS points were tracked near this address.".format(maximum, total, maximum/total))
    print("Another {} coordinate points were tracked within {} Meters of the address".format(proximity, distance*1000))
    if complete:
        address = "{} {}, {} {}".format(street, number, zip_code, city)
        print(address)
    if not complete:
        address = "{} {} {}".format(quarter, city, country)
        print("My best guess is: {}".format(address))

In [25]:
print("The address considering nightly data points:\n")
dict_comprehension(wohnort_night, maximum_night, total_night, range_night)
print("\nThe address considering all data points:\n")
dict_comprehension(wohnort_allday, maximum_allday, total_allday, range_allday)

The address considering nightly data points:

The GPS data was analyzed. 280 out of 562 (49.82%) GPS points were tracked near this address.
Another 57 coordinate points were tracked within 150.0 Meters of the address
Schönbrunner Straße 277, 1120 Wien

The address considering all data points:

The GPS data was analyzed. 448 out of 1084 (41.33%) GPS points were tracked near this address.
Another 83 coordinate points were tracked within 150.0 Meters of the address
Schönbrunner Straße 277, 1120 Wien


In [26]:
from folium.features import CustomIcon
karte = folium.Map(location=[lat_max_allday, lon_max_allday], zoom_start = "19")
# Icon made by Dave Gandy from www.flaticon.com
icon_image = "https://image.flaticon.com/icons/svg/25/25694.svg"
icon = CustomIcon(
    icon_image,
    icon_size=(38, 95),
    icon_anchor=(22, 65),
    popup_anchor=(-1, -35)
)        
folium.Marker(location = [coordinates_address_allday.latitude, coordinates_address_allday.longitude], 
              icon = icon,
              popup = "Home: {}".format(address)).add_to(karte)
karte 